# Python for High Performance Computing
# Introduction
<hr style="border: solid 4px green ">
<br>
<center><img src="../../images/arc_logo.png"; style="float: center; width: 20%"></center>
<br>
## http://www.arc.ox.ac.uk
## support@arc.ox.ac.uk

## What you need
<hr style="border: solid 4px green">

### Intended audience
* intermediate to advanced Python programmers who develop performance-critical applications
<br><br>

### Aims
* explore the use of Python core packages for scientific computing
* gain experience with a number of Python solutions for parallel processing
<br><br>

### Material
* laptop (Windows, Linux, MacOS)
* course material (notebooks and source code)
* download from https://github.com/mcduta/python-hpc
<br><br>

### Software
* Python **version 2.7**
  * the `anaconda2` distribution is recommended (https://www.continuum.io/downloads)
  * the Intel distribution for Python is marginally better and will be the recommendation in future
* compilers (C and Fortran)
  * Linux comes with `gcc`, but `gfortran` needs to be installed separately
  * MacOS has `gcc` but that is an alias for `clang` (LLVM front-end) -- both `gcc` and `gfortran` must be installed separately
  * Windows needs both compilers, easiest installable in `Cygwin`
* a SSH client and an X client
  * Linux has everything out of the box
  * MacOS has the SSH client installed but needs the X client (*e.g.* XQuartz)
  * Windows needs both installed (MobaXterm is recommended)

## High Performance Computing
<hr style="border: solid 4px green ">

### Development of  scientific applications
* make it *run*
  * software in working state, and produces correct results
  * explore the scientific problem and spot (major) early-stage software design issues
* make it *right*
  * achieve a solid software design of the program is solid
  * separate application into independent and cohesive units that are easy to maintain
* make it *fast*
  * optimize the parts of the program that are not fast enough
<br><br>

### **H**igh **P**erformace **C**omputing (HPC)
* the practice of programming and running intensive applications efficiently on performance hardware
* applications are (usually) run in parallel and stress all three components that define performance
  * CPU
  * I/O
  * network
<br><br>

### Computing performance determines the scale and complexity of the scientific numerical problems we can solve

## High Performance Computing (cont'd)
<hr style="border: solid 4px green ">

### How fast is fast enough?
* a PC can deliver tens of Gflops (flops = floating point operations per second) *but*
  * tens of millions is a lot of flops but that may or may not be enough
  * **N.B.**: theoretical peak performance $\neq$ application performance
<br><br>

### Extreme example: short range weather forecast
* prediction for next day(s) must be delivered in less than 1 day
* the MetOffice current models require ~1 Pflops (= 1M x what a PC can deliver)
* this is a few models, ensemble forecast is even more expensive
<br><br>

### More moderate examples: university research 
* materials science
* computational physics (galaxy formation, plasma modelling)
* earth physics (climate, oceans, earthquakes)
* engineering (fluids, combustion, structures)
* biochemistry (cell membrane mechanisms, drug discovery)

## Why Python?
<hr style="border: solid 4px green ">

### Python is a *high level* programming language
* conventional syntax (close to pseudocode and mathematics)
* safe semantics (any violation produces errors)
* supports both procedural and object-oriented programming
* easy to prototype
<br><br>

### Python is a *high-productivity* language
* simple to learn and use
* you spend more time thinking about what code does rather than how to write it
* relatively easy to do relatively hard things
<br><br>

### Python has rich *scientific computing functionality*
* numerical and scientific libraries
* domain-specific packages
* powerful plotting capabilities
* excellent for gluing existing Fortran/C/C++ code to create a processing pipeline
<br><br>

### Python has *excellent* community support

## Python and HPC
<hr style="border: solid 4px green; ">

### Python is <font color=green>fast</font>
* for writing, testing and developing code
<br><br>

### Python is <font color=red>slow</font>
* for executing research/scientific code, *espec.* repeated execution of low-level tasks
<br><br>

### Performance gap: what makes Python fast (for development) makes Python slow (for execution)
* Python is *interpreted*
* Python is *dynamic-typed*
* Python is *serial*
<br><br>

### The aim of this course
* explore ways in which the *performance gap* can be bridged

## Cause #1: Python is slow because it is interpreted
<hr style="border: solid 4px green; ">

### "Interpreted languages" are slow compare with "compiled languages"
* in fact, interpretation and compilation are properties of the *implementation* of a language rather than of the *language* itself
<br><br>

### Compiled languages
* a compiler translates the source code directly into machine code
* the machine code is directly executed by the target machine and is thus specific to
  * a target processor
  * an operating system
* the compiler applies a host of optimisations in the translation to machine code
<br><br>

### Interpreted languages
* the interpreter executes the source code
* the interpreter is specific to
  * a target processor
  * an operating system
* the interpreter has a "narrow" view of the source (one line at a time) and cannot apply optimisations

## Cause #1: Python is slow because it is interpreted (cont'd)
<hr style="border: solid 4px green; ">

## How slow?
<br><br>

### A lot!
* a pure Python implementation could be orders of magnitude slower than its Fortran or C equivalent
* even the best pure Python implementation (using `NumPy`) can be slower
<br><br>

### Example
Define a Python function using `NumPy` arrays (fast Python) that is bound to perform well.

In [1]:
import numpy as np
def my_func_python (x, y):
    return np.sin (x**2 + y**2)

Now, define its Fortran equivalent (using the `fortranmagic` module).

In [2]:
%load_ext fortranmagic

/usr/local/anaconda2/lib/python2.7/site-packages/IPython/utils/path.py:265: UserWarning: get_ipython_cache_dir has moved to the IPython.paths module
  warn("get_ipython_cache_dir has moved to the IPython.paths module")


In [3]:
%%fortran -v --opt="-O3"
       subroutine my_func_fortran (x, y, z)
            real, intent(in) :: x(:), y(:)
            real, intent(out) :: z(size(x))
            ! using vector operations
            z = sin (x**2 + y**2)
       end subroutine


Ok. The following fortran objects are ready to use: my_func_fortran


And now, we time both approaches for large arrays.

In [4]:
x = np.random.normal (size=100000000)
y = np.random.normal (size=100000000)
% timeit my_func_python (x, y)
% timeit my_func_fortran (x, y)

1 loop, best of 3: 2.86 s per loop
1 loop, best of 3: 1.43 s per loop


### The `NumPy` implementation
* is *the best of* "native" Python performance
* is slower than Fortran
* but a pure Python implementation (no `NumPy` arrays) is even slower... a lot slower
<br><br>

### Furthermore
* `NumPy` is limited to a single thread of execution
* Fortran can be made even faster
  * serial execution: extra compiler optimisation
  * parallel execution: OpenMP threading

## Cause #2: Python is dynamic
<hr style="border: solid 4px green; ">

### Static typing
* type of a variable declared before the variable is used
* type checking is done at compile time
```c
int num;    // explicit declaration
num = 10;   // use the variable
num = 10.0; // this will throw an error
```
<br><br>

### Dynamic typing
* type of a variables is inferred from their use
* type checking is done at run time
```python
num = 10   # just use the variable
num = 10.0 # this is valid
```
<br><br>

### Python type-checking overhead
* every time a variable is used there is a check
* this overhead becomes significant in repeated operations (*e.g.* a loop)

## Cause #2: Python is dynamic (cont'd)
<hr style="border: solid 4px green; ">

### Python treats every variable and function call like a box of chocolates
* you never know what you're gonna get
* so checks are applied at all steps
<br><br>

### A simple integer addition
```python
>>> 1+2
3
```

### is interpreted and executed as follows
* the operation

<table border="0">
  <tr>
    <th><img src="./images/pyobj1.png"; style="float: center; width: 3cm"><th>
    <th><font size="10">+</font></th>
    <th><img src="./images/pyobj2.png"; style="float: center; width: 3cm"><th>
  </tr>
</table>

* is operand 1 of same type as operand 2?
  * yes: try to do something with them
    * is the operation requested valid and callable?
      * yes: call "add" on the two integer arguments and generate a new Python object to contain the result
      <img src="./images/pyobj3.png"; style="float: center; width: 3cm">
      * no: throw an error
  * no: throw an error
<br><br>

### Interpreted computing is safe but there is a lot of overhead!

## Cause #2: Python is dynamic (cont'd)
<hr style="border: solid 4px green; ">

### A simple function call
```python
>>> def f(a, b):
...    return a+b
...
>>> f(1, 2)
3
```

### is executed in the following way
* the function call

<table border="0">
  <tr>
    <th>Argument 1</th>
    <th> </th>
    <th>Argument 2</th>
    <th> </th>
    <th>Argument list</th>
    <th> </th>
    <th>Function Object</th>
    <th> </th>
    <th>Return object</th>
  </tr>
  <tr>
    <td><img src="./images/pyobj1.png"; style="float: center; width: 3cm"></td>
    <td><font size="10">,</font></td>
    <td><img src="./images/pyobj2.png"; style="float: center; width: 3cm"></td>
    <td><font size="10">></font></td>
    <td><img src="./images/pyobj5.png"; style="float: center; width: 3cm"></td>
    <td><font size="10">></font></td>
    <td><img src="./images/pyobj6.png"; style="float: center; width: 3cm"></td>
    <td><font size="10">></font></td>
    <td><img src="./images/pyobj3.png"; style="float: center; width: 3cm"></td>
  </tr>
</table>


* is `f` a callable object?
  * yes
    * form list from arguments
    * pass argument list to callable
    * the callable object unpacks the argument list and applies code (see previous slide)
    * if successful, form new object and return it
  * no: throw an error
<br><br>

### Function call is safe but there is a lot of overhead!
* more than C or Fortran function calls, that is

## Cause #2: Python is dynamic (cont'd)
<hr style="border: solid 4px green; ">

### Function call overhead can be measured

In [5]:
import time

def f():
    pass # null operation: do nothing

n = 10**6 # a large number of times
npass = 10

t1 = time.time()
for ipass in xrange (npass):
    for i in xrange (n):
        pass
t1 = time.time() - t1
t1/= float (npass)
print "time (no function call) = ", t1, "sec"

t2 = time.time()
for ipass in xrange (npass):
    for i in xrange (n):
        f()
t2 = time.time() - t2
t2/= float(npass)
print "time (function call)    = ", t2, "sec"
print "function call overhead  = ", (t2-t1) / float(n)*1.e+9, "nsec"

time (no function call) =  0.0657330036163 sec
time (function call)    =  0.191447997093 sec
function call overhead  =  125.714993477 nsec


## Cause #3: Python is serial
<hr style="border: solid 4px green; ">

### The culprit
The **G**lobal **I**nterpreter **L**ock (GIL) is a major limitation of cPython
<br><br>

### GIL
* it is a mutex = a **m**utual **ex**clusion property in concurrency control with the purpose of preventing *race conditions*
* it prevents native threads from executing Python bytecodes concurrently -- only a single thread can acquire a lock on a Python object or C API at any one time
* it *is* necessary because the interpreter is not thread-safe and GIL protects access to current thread state and heap allocated objects
* but it also prevents Python programs from taking full advantage of multiprocessor systems
* effectively the language's most difficult technical challenge

## Making Python faster
<hr style="border: solid 4px green; ">

### An active area of research, a jungle of solutions out there!
<br><br>


### There are *very* many solutions for making Python execution
* faster in serial
* parallel
<br><br>

### All solutions address one or more of the causes for which Python is slow

## Profiling Python code
<hr style="border: solid 4px green; ">

### Identify what to optimise
Performance-critical parts of Python code are identified through *profiling*.

### Python tools for profiling
* `profile` and `cProfile` modules
  * standard run time analysis and function call stack
  * `profile` (pure Python) has heavier overheads
* `line_profiler` and `kernprof`
  * a set of tools for line by line code timing
* `memory_profiler`
  * a tool for line by line memory code footprint
* `IPython` timers
  * the `timeit` function is useful for timing functions interactively

> *Warning*: Python development tools (debuggers, profilers) lag behind compiled code options

## Faster Python: approach #1
<hr style="border: solid 4px green; ">

### Make *everything* faster
* `PyPy`
  * fast, compliant implementation of the Python language
  * alternative to CPython
  * compatible with Python versions 2.7 and 3.3
* `Nuitka`
  * ahead-of-time compiler to generate fast executing standalone programs from Python source
  * Python to C++ source-to-source transformation
  * C++ source compiled and optimised
<br><br>

#### Pros
* drop-in replacement for pure Python code
<br><br>

#### Cons
* uncertainty about the future of the solutions
* no perfect compatibility (*e.g.* C extensions in `PyPy`)
* no support for some Python modules
* no fine control over development
* no parallelism

> A comprehensive list of Python "compilers" at http://compilers.pydata.org/

## Faster Python: approach #2
<hr style="border: solid 4px green; ">

### Make the *critical* parts faster
* use the right Python module
  * general purpose, *e.g.* `NumPy` and `SciPy`
  * specialised modules, *e.g.* `Pandas`, `SciKits`
* extend Python
  * write functions in C/C++/Fortran for speed
  * use existing C/C++/Fortran libraries for functionality
  * extensions/libraries can be parallel, *e.g.* `NumPy` uses Intel MKL
* parallelise Python code
* use **J**ust **I**n **T**ime (JIT) compiling
<br><br>

#### Pros
* fine control over the technical solution
* potentially sustainable solution
  * use existing software
  * benefit from community support
  * spread risks
<br><br>

#### Cons
* development effort
* requires learning new techniques

## Faster Python: the easy way (staying with Python)
<hr style="border: solid 4px green; ">

### General good practice: use specialised Python modules
* `NumPy` and `SciPy`
  * contiguous array N dimensional implementation
  * very fast for strided operations of arrays
  * fundamental methods for scientific computing
* `pandas`
  * package for data frames (or tables)
  * very fast analytics on arrays
* `pytables`
  * fast structured hierarchical tables (such as `hdf5`)
  * good for out of core calculations and queries on large data
* `scikit-learn`
    * package for machine learning, data mining and data analysis

## Faster Python: the easy way (staying with Python) cont'd
<hr style="border: solid 4px green; ">

### JIT compilation
* Python code translated to machine code on the fly
* compiler optimisations are applied
* "lazy" JIT is the easiest option:
  * functions are compiled first time they are called (slow execution)
  * and use cached machine code at subsequent calls (fast execution)
<br><br>

### JIT compilers
* `numba`
  * package for Python code decorations to allow JIT compilation to native machine instructions
* `Theano`
  * library that allows you to define, optimise, and evaluate efficiently mathematical expressions involving multi-dimensional arrays
  * transparent use of a GPU

## Faster Python: the easy way (staying with Python) cont'd
<hr style="border: solid 4px green; ">

### Other options
* `Cython`
  * Python compiler (Python code to standalone executable)
  * decorate performance critical Python functions
  * generate tuned C code, then compiled to library of functions usable from Python
  * a very maintainable solution, easily links to other (hand-written) C/C++/Fortran code
  * easy to multi-thread using OpenMP
  * by far the preferred tool today
* `multiprocess`
  * launches and controls concurrent processes on a single host

## Faster Python: the hard way
<hr style="border: solid 4px green; ">

### Writing C/C++/Fortran functionality
* the `ctypes` package
  * hand-write C functions and wrap them for Python
* the `f2py` tool
  * hand-wite Fortran functions and wrap them for Python
* SWIG (**S**implified **W**rapper and **I**nterface **G**enerator)
  * generates Python modules from C/C++ code starting from header files
<br><br>

### Scaling Python beyond a single host (distributed computing libraries)
* `mpi4py` -- MPI Python wrapper (fastest and most complete)
* `disco` -- Python Hadoop-like framework
* `pathos` -- framework for computing on heterogeneous resources
* Global Arrays bindings -- a shared memory interface to distributed computing
<br><br>

### Domain specific libraries
* `petsc4py` -- Python bindings for PETSc (**P**ortable, **E**xtensible **T**oolkit for **S**cientific **c**omputations)
* `slepc4py`-- Python bindings for SLEPc (**S**calable **L**ibrary for **E**igenvalue **P**roblem **c**omputations)
* `tao4py` -- Python bindings for large-scale optimisation problems using TAO (**T**oolkit for **A**dvanced **O**ptimization)
* `pyTrilinos`: Trilinos wrappers

## This course will cover
<hr style="border: solid 4px green">

### Day 1: focus on <span style="font-family: Courier New, Courier, monospace;">NumPy</span> and serial solutions
* `numpy`
  * tools for manipulating arrays
* `scipy`
  * high-level scientific routines for common algorithms,
    *e.g.* numerical integration, optimisation, Fourier transform
* `matplotlib`
  * plotting in 2D and 3D
* serial code acceleration
  * `numpy`
  * `numba`
  * `ctypes` (C source)
  * `f2py` (Fortran source)
  * other tools (`numpy.blitz`, `numpy.inline`, `numba`)
* multithreading
  * `numba`
<br><br>

### Day 2: focus on parallel solutions
* multithreading C/Fortran source
  * combining `ctypes` and `f2py` with OpenMP
* `cython`
* `multiprocessing`
  * multicore task parallelism
* `mpi4pi`
  * **M**essage **P**assing **I**nterface (MPI) parallel programming

## Test problems
<hr style="border: solid 4px green; ">

### Calculation of the p-norm of a vector
<br><br>

### Trapezium integration
<br><br>

### Monte--Carlo integration
<br><br>

### The 2D heat equation

## Calculation of the p-norm of a vector
<hr style="border: solid 4px green; ">

The $p$-norm of a vector $x$ is
$$
||x||_p =\left( \sum_{i=1}^{n} |x_i|^p \right)^{\frac{1}{p}}
$$
<br>
* a generalisation of the Euclidian norm ($p=2$)
* $p=3$ makes computation expensive and comparison of implementations meaningful

## Monte--Carlo integration
<hr style="border: solid 4px green; ">

### Computing $\pi$ using Monte-Carlo integration
Computing high-dimensional definite integrals with complicated boundaries was the first use of the Monte-Carlo method.  Using this approach, $\pi$ can be computed as the surface integral over the unit circle of a constant unit integrand.

The algorithm of computing $\pi$ is the area of the unit circle in the 2D plane is to "throw darts" and to count those that fall in the circle.  More specifically,
* generate a number of $N$ (uniformly distributed) points in the 2D plane ;
* count the number $N_{\text{in}}$ of points in the unit circle;
* compute the approximation as $\pi\approx N/N_{\text{in}}$.

Exploiting the symmetry of the unit circle, only the area of a single quadrant of the circle can be computer.  Thus, the random points are generated with coordinates in the $[0, 1)$ interval and the estimated value of $pi$ is
$$
\pi_{approx} = 4 \frac{N}{N_{\text{in}}} \longrightarrow \pi \ \ \ \ \ \ \ \ \ \ \text{as} \ \ \ \ \ \ \ N \longrightarrow \infty
$$

<img src="./images/darts.png"; style="float: center; width: 60%"; >

> *Observation*: the estimated value converges to $\pi$ very slowly with $N$>

## Test problems
<hr style="border: solid 4px green; ">

### Integration by the trapezium rule
You are asked to evaluate a definite integral using the trapezium rule.  The integral is
$$
\int_{0}^{\sqrt{\pi}} x\cdot \sin x^2\ dx \ =\ \left. -\frac{1}{2}\cdot \cos{x^2} \right|^\sqrt{\pi}_{0} \ =\ 1
$$

The trapezium rule for evaluating the integral
$$
\int_{a}^{b} f(x) dx
$$
of a continuous function $f$ approximates the area between the graph of $f(x)$ and the $x$-axis as a series of trapeziums.  Thus, the value of the integral is simply approximated as the sum of the areas of the trapeziums.

Assuming the interval $x \in [a, b]$ is discretised using $N$ intervals of equally spaced points of length $h=(b-a)/N$, the vertices of the trapeziums on the $x$-axis are given by
$$
x_i\ =\ a + i\cdot h,\ \ \ \ i=0,\dots,N
$$

The areas of the trapeziums formed by $x_{i}$, $x_{i+1}$ and $f(x_{i})$, $f(x_{i+1})$ add up to
$$
\int_{a}^{b} f(x) dx \approx \sum_{0}^{N-1} \frac{h\cdot ( f(x_{i}) + f(x_{i+1}) )}{2} = \frac{f(a)}{2} + f(x_1) + f(x_2) + \dots + f(x_{N-1}) + \frac{f(b)}{2}
$$

<img src="./images/integral.png"; style="float: center; width: 60%"; >

## The 2D heat equation
<hr style="border: solid 4px green; ">

### The maths
The problem is to find the time-varying temperature distribution across a plate given an initial distribution and a fixed temperature around the edges.  Mathematically, this means solving the equation

$$\frac{\partial u}{\partial t}=\frac{\partial^2 u}{\partial x^2}+\frac{\partial^2 u}{\partial y^2}$$

subject to
* the initial condition $u(x,y,0)=u_0(x,y)$ and
* the boundary condition $u(x,y,t)=0$ on the boundary.

Assume the domain of the physical problem to be the unit square $0\leq x,y\leq 1$ with the initial conditions

$$u_0(x,y)=\sin\pi x\cdot\sin\pi y$$

With the above, the analytic solution to the mathematical problem is

$$u(x,y,t)=\sin\pi x\cdot\sin\pi y\cdot e^{-2\pi^2 t}$$

which is useful to validate the numerical solution in the Python implementation.

### The numerical solution
The numerical solution used in this example is via the **F**inite **D**ifference (FD) solution.  Thus, both the space coordinates $0\leq x,y\leq 1$ and time $t\geq 0$ are discretised and the continuous solution $u$ is approximated by a discrete equivalent (an array), which is advanced in time from the initial conditions to a chosen final time value.  In summary, the FD solution means
* sample the 2D domain at equidistant points at coordinates $x_i=i\Delta x$ and $y_j=j\Delta y$
* sample time at points $t_n=n\Delta t$
* compute the discrete values $u^n_{i,j}$ corresponsing to the $x_i$, $y_j$ and $t_n$
* assuming $\Delta x=\Delta y$, the numerical solution is produced by the *time-marching scheme*

$$u^{n+1}_{i,j}=u^{n}_{i,j}+\nu \left( u^{n}_{i+1,j}+u^{n}_{i-1,j}+u^{n}_{i,j+1}+u^{n}_{i,j-1}\ -\ 4u^{n}_{i,j} \right)$$

where $\nu=\frac{\Delta t}{\Delta x^2}\leq 0.25$ for numerical stability.

This numerical scheme results in the 6 point stencil involving *each* discrete point $(i, j)$ at time $t_{n+1}$ is updated from *five* values at time $t_n$ (the same point plus its neighbours).  Diagramatically, this is represented as
<br><br>

<table border="0">
  <tr>
    <td><center>time step $n$</center></td>
    <td><center>time step $n+1$</center></td>
  </tr>
  <tr>
    <td><img src="./images/untxt.png"; style="float: center; width: 40%"></td>
    <td><img src="./images/unp1txt.png"; style="float: center; width: 40%"></td>
  </tr>
</table>

### The numerical algorithm
The algorithm is
* start with initial conditions
* for each time step
  * for each space point, use current solution $u^{n}$ to compute next step $u^{n+1}$
  * apply boundary conditions
  * swap current solution with updated "next-step" solution

<img src="../../images/reusematerial.png"; style="float: center; width: 90"; >